In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

import json
import os

os.chdir("Test Data")
random_state = 1

airfoil_train = pd.read_csv("airfoil_train.csv")
airfoil_test = pd.read_csv("airfoil_test.csv")

bike_train = pd.read_csv("bike_train.csv")
bike_test = pd.read_csv("bike_test.csv")

energy_train = pd.read_csv("Energy_train.csv")
energy_test = pd.read_csv("Energy_test.csv")

lending_train = pd.read_csv("Lending_Club_train.csv")
lending_test = pd.read_csv("Lending_Club_test.csv")

california_train = pd.read_csv("california_train.csv")
california_test = pd.read_csv("california_test.csv")


In [ ]:
def test_function(results_df, train, test, model, model_name, dataset_name):
    X_train = train.drop(['y'], axis = 1)
    y_train = train.y
    X_test = test.drop(['y'], axis = 1)
    y_test = test.y
    
    start_time = time.time()
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    total_time = (time.time()-start_time)
    results_df.loc[len(results_df)] = [dataset_name, model_name, mse, total_time]
    
    return results_df


In [ ]:
import time
def train_hyperparameters(mod, grid, output_name, train):
    start_time = time.time()
    X_train = train.drop(['y'], axis = 1)
    y_train = train.y
    
    cv = KFold(n_splits=5, shuffle=True, random_state=1)

    grid_search = GridSearchCV(estimator=mod, param_grid=grid, 
                               cv=cv, n_jobs=-1, verbose=1, scoring = 'neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best score:", -1*grid_search.best_score_)
    print("Best params: ", best_params)
    print("Time: ", time.time()-start_time)

    with open('Hyperparameters/'+output_name+'.json', 'w') as f:
        json.dump(best_params, f)

# Random Forest

In [ ]:
param_grid = {
    'n_estimators': [300],
    'max_depth': [12, 15, 18, 21, 24, 27],
    'max_features': ["sqrt", "log2", 1.0, 0.5],
    
}
rf = RandomForestRegressor(random_state = 1)
train_hyperparameters(rf, param_grid, "rf_airfoil", airfoil_train)
train_hyperparameters(rf, param_grid, "rf_bike", bike_train)
train_hyperparameters(rf, param_grid, "rf_energy", energy_train)
train_hyperparameters(rf, param_grid, "rf_lending", lending_train)
train_hyperparameters(rf, param_grid, "rf_california", california_train)


## Testing random forest

In [ ]:
def rf_test_function(results_df, train, test, dataset_name, parameters_name):
    with open('Hyperparameters/' + parameters_name + '.json') as f:
        best_params = json.load(f)
    best_rf = RandomForestRegressor(**best_params, random_state = 1)
    return test_function(results_df, train, test, best_rf, "Random Forest", dataset_name)



In [ ]:
results_df = pd.DataFrame(columns=['Dataset', 'Model', 'Model MSE', 'Time'])

results_df = rf_test_function(results_df, airfoil_train, airfoil_test, "airfoil", "rf_airfoil")
results_df = rf_test_function(results_df, bike_train, bike_test, "bike", "rf_bike")
results_df = rf_test_function(results_df, energy_train, energy_test, "energy", "rf_energy")
results_df = rf_test_function(results_df, lending_train, lending_test, "lending", "rf_lending")
results_df = rf_test_function(results_df, california_train, california_test, "california", "rf_california")


In [ ]:
results_df

In [ ]:
results_df.to_csv("RF Test Results real.csv")


# XGBoost

In [ ]:
param_grid = {
    'max_depth': [4,6,8],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'reg_lambda':[0.1, 1, 10],
    'n_estimators':[100, 500, 1000],
    'subsample': [0.6, 0.8, 1.0],
}

xgboost = xgb.XGBRegressor(random_state=1)
train_hyperparameters(xgboost, param_grid, "xgb_airfoil", airfoil_train)
train_hyperparameters(xgboost, param_grid, "xgb_bike", bike_train)
train_hyperparameters(xgboost, param_grid, "xgb_energy", energy_train)
train_hyperparameters(xgboost, param_grid, "xgb_lending", lending_train)
train_hyperparameters(xgboost, param_grid, "xgb_california", california_train)



In [ ]:
def xgb_test_function(results_df, train, test, dataset_name, parameters_name):
    with open('Hyperparameters/' + parameters_name + '.json') as f:
        best_params = json.load(f)
    best_rf = xgb.XGBRegressor(**best_params, random_state = 1)
    return test_function(results_df, train, test, best_rf, "Xtreme Gradient Boosting", dataset_name)



In [ ]:
results_df = pd.DataFrame(columns=['Dataset', 'Model', 'Model MSE', 'Time'])
results_df = xgb_test_function(results_df, airfoil_train, airfoil_test, "airfoil", "xgb_airfoil")
results_df = xgb_test_function(results_df, bike_train, bike_test, "bike", "xgb_bike")
results_df = xgb_test_function(results_df, energy_train, energy_test, "energy", "xgb_energy")
results_df = xgb_test_function(results_df, lending_train, lending_test, "lending", "xgb_lending")
results_df = xgb_test_function(results_df, california_train, california_test, "california", "xgb_california")




In [ ]:
results_df.to_csv("XGB Test Results real.csv")

